In [1]:
from seleniumwire import webdriver
from selenium.webdriver.common.by import By
import requests,os,sys,json,re,zlib,brotli
from bs4 import BeautifulSoup
from time import sleep
from pprint import pprint
import pandas as pd


In [2]:
from leagues.models import Season,Competition
from base_app.decorators import catch_iteration_errors
from base_app.helpers import fetch_configurations
from base_app.models import ConfigItems

In [3]:
CONFIG = fetch_configurations()

In [ ]:
all_comps_sa = Competition.objects.filter(event_data_available=True)

In [ ]:
all_comps_sa[0].name_scoresaway

## Navigate to page, get all values and links in the select box. SIMPLE

In [15]:
def is_recent(entry, threshold=int(CONFIG.get('START_YEAR'))):
    try:
        # Split by '/' or just wrap in a list if single year
        years = [int(y) for y in str(entry).split('/') if y.isdigit()]
        # Keep if any year is >= threshold
        return any(y >= threshold for y in years)
    except ValueError:
        return False

In [ ]:
@catch_iteration_errors
def fetch_event_season_urls(index,comp):
    driver = webdriver.Chrome()
    driver.get(comp.event_data_url)
    select_box = driver.find_element(By.CSS_SELECTOR, "#season-select")
    links = {season.get_attribute('innerText') : f"https://www.scoresway.com{season.get_property('value')}" for season in select_box.find_elements(By.CSS_SELECTOR, "option")}
    print(links)
    driver.quit()
    return links

for i,comp in enumerate(all_comps_sa,1) :
    link_dicts = fetch_event_season_urls(i,comp)
    if comp.name_scoresaway is None :
        print("nahi hua : ",comp.__dict__)
        print()
        continue
    seasons_considered = {yr:value for yr,value in link_dicts.items() if is_recent(yr)}
    for season_name, event_url in seasons_considered.items():
        Season.objects.create(competition=comp,name=season_name,season_event_url=event_url)
    print()
    # break

## FOR FOTMOB IT"LL WORK SAME

In [4]:
all_comps_fm = Competition.objects.filter(shot_data_available=True)

In [8]:
# @catch_iteration_errors
# def fetch_shot_season_urls(index,comp):
#     driver = webdriver.Chrome()
#     driver.get(comp.shot_data_url)
    
#     for request in driver.requests:
#         if request.response and "https://www.fotmob.com/api/leagues" in request.url:
#             encoding = request.response.headers.get('Content-Encoding', '')
#             raw_body = request.response.body
    
#             if encoding == 'br':
#                 body = brotli.decompress(raw_body).decode('utf-8')
#             elif encoding == 'gzip':
#                 body = zlib.decompress(raw_body, zlib.MAX_WBITS | 16).decode('utf-8')
#             elif encoding == 'deflate':
#                 body = zlib.decompress(raw_body).decode('utf-8')
#             else:
#                 body = raw_body.decode('utf-8')
    
#     all_seasons = [x for x in json.loads(body).get('allAvailableSeasons') if is_recent_new(x)]
#     all_season_urls = {}
#     print("Working for :",comp.name_fotmob)
#     for season in all_seasons:
#         if season not in all_season_urls:
#             current_obj = Season.objects.get(competition=comp,name=season)
#             link = f'{comp.shot_data_url}?season={season.replace(r"/","-").replace(" ","+")}'
#             current_obj.season_shot_url = link
#             current_obj.save()
#     driver.quit()

# for i,comp in enumerate(all_comps_fm,1) :
#     link_dicts = fetch_shot_season_urls(i,comp)


In [5]:
jap = next(x for x in all_comps_fm if x.name_fotmob=='J. League')
mex = next(x for x in all_comps_fm if x.name_fotmob=='Liga MX')

In [20]:
driver = webdriver.Chrome()
driver.get(mex.shot_data_url)

for request in driver.requests:
    if request.response and "https://www.fotmob.com/api/leagues" in request.url:
        encoding = request.response.headers.get('Content-Encoding', '')
        raw_body = request.response.body
        print(encoding)

        if encoding == 'br':
            body = brotli.decompress(raw_body).decode('utf-8')
        elif encoding == 'gzip':
            body = zlib.decompress(raw_body, zlib.MAX_WBITS | 16).decode('utf-8')
        elif encoding == 'deflate':
            body = zlib.decompress(raw_body).decode('utf-8')
        else:
            body = raw_body.decode('utf-8')
        print(body)
driver.quit()
all_seasons = [x for x in json.loads(body).get('allAvailableSeasons') if is_recent(x)]
# all_season_urls = {}

br
{"tabs":["overview","table","matches","stats","transfers"],"allAvailableSeasons":["2024/2025 - Clausura","2024/2025 - Apertura","2023/2024 - Clausura","2023/2024 - Apertura","2022/2023 - Clausura","2022/2023 - Apertura","2021/2022 - Clausura","2021/2022 - Apertura","2020/2021 - Clausura","2020/2021 - Apertura","2019/2020 - Clausura","2019/2020 - Apertura","2018/2019 - Clausura","2018/2019 - Apertura","2017/2018 - Clausura","2017/2018 - Apertura","2016/2017 - Clausura","2016/2017 - Apertura","2015/2016 - Clausura","2015/2016 - Apertura","2014/2015 - Clausura","2014/2015 - Apertura","2013/2014 - Clausura","2013/2014 - Apertura","2012/2013 - Clausura","2012/2013 - Apertura","2011/2012 - Clausura","2011/2012 - Apertura","2010/2011 - Clausura","2010/2011 - Apertura"],"details":{"id":230,"type":"league","name":"Liga MX","selectedSeason":"2024/2025 - Clausura","latestSeason":"2024/2025 - Clausura","shortName":"Liga MX","country":"MEX","faqJSONLD":null,"breadcrumbJSONLD":{"@context":"https:

In [18]:
[x for x in json.loads(body).get('allAvailableSeasons') if is_recent(x)]

['2024/2025 - Clausura',
 '2024/2025 - Apertura',
 '2023/2024 - Clausura',
 '2023/2024 - Apertura',
 '2022/2023 - Clausura',
 '2022/2023 - Apertura',
 '2021/2022 - Clausura',
 '2021/2022 - Apertura',
 '2020/2021 - Clausura',
 '2020/2021 - Apertura',
 '2019/2020 - Clausura',
 '2019/2020 - Apertura',
 '2018/2019 - Clausura',
 '2018/2019 - Apertura']

In [26]:
season_links = {}
for x in all_seasons:
    season_type = x.split('-')[-1].strip()
    if season_type not in season_links :
        season_links[season_type] = []
    season_links[season_type].append((x,f'{mex.shot_data_url}?season={x.replace(r"/","-").replace(" ","+")}'))

In [27]:
pprint(season_links)

{'Apertura': [('2024/2025 - Apertura',
               'https://www.fotmob.com/leagues/230/overview/liga-mx?season=2024-2025+-+Apertura'),
              ('2023/2024 - Apertura',
               'https://www.fotmob.com/leagues/230/overview/liga-mx?season=2023-2024+-+Apertura'),
              ('2022/2023 - Apertura',
               'https://www.fotmob.com/leagues/230/overview/liga-mx?season=2022-2023+-+Apertura'),
              ('2021/2022 - Apertura',
               'https://www.fotmob.com/leagues/230/overview/liga-mx?season=2021-2022+-+Apertura'),
              ('2020/2021 - Apertura',
               'https://www.fotmob.com/leagues/230/overview/liga-mx?season=2020-2021+-+Apertura'),
              ('2019/2020 - Apertura',
               'https://www.fotmob.com/leagues/230/overview/liga-mx?season=2019-2020+-+Apertura'),
              ('2018/2019 - Apertura',
               'https://www.fotmob.com/leagues/230/overview/liga-mx?season=2018-2019+-+Apertura')],
 'Clausura': [('2024/2025 - Claus

In [6]:
mexico_items = Season.objects.filter(competition__id=27)

In [37]:
for i,item in enumerate(mexico_items):
    name,link = season_links['Clausura'][i]
    print(f"""
            competition = {item.competition}
            name = {item.name}
            name_fotmob = {name}
            season_event_url = {item.season_event_url}
            season_shot_url = {link}
            """)
    Season.objects.create(
            competition = item.competition,
            name = item.name,
            name_fotmob = name,
            season_event_url = item.season_event_url,
            season_shot_url = link)

           # item.save())
    # item.name_fotmob = f"{item.name_fotmob} - Apertura"
    # item.save()
    # print(item)
    # break


            competition = Liga MX - Mexico
            name = 2024/2025
            name_fotmob = 2024/2025 - Clausura
            season_event_url = https://www.scoresway.com/en_GB/soccer/liga-mx-2024-2025/dlxyh3nrdtg02p9ohd5gn6bdg/fixtures
            season_shot_url = https://www.fotmob.com/leagues/230/overview/liga-mx?season=2024-2025+-+Clausura
            

            competition = Liga MX - Mexico
            name = 2023/2024
            name_fotmob = 2023/2024 - Clausura
            season_event_url = https://www.scoresway.com/en_GB/soccer/liga-mx-2023-2024/cgr8k7g0pwtlbzca75gxq1g5w/results
            season_shot_url = https://www.fotmob.com/leagues/230/overview/liga-mx?season=2023-2024+-+Clausura
            

            competition = Liga MX - Mexico
            name = 2022/2023
            name_fotmob = 2022/2023 - Clausura
            season_event_url = https://www.scoresway.com/en_GB/soccer/liga-mx-2022-2023/ei37yy9qltzgkbd3c15hq1d04/results
            season_shot_url

IndexError: list index out of range